In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time
import datetime as dt
import re

from IPython.display import display, HTML, clear_output
from IPython.core.interactiveshell import InteractiveShell
import ipywidgets as widgets

from matplotlib import pyplot as plt
import seaborn as sns

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:79% !important; }</style>"))

# easier-to-read notebook:
from IPython.display import display, HTML, clear_output
pd.options.display.max_columns = 70
pd.options.display.max_rows = 500
%load_ext autoreload
%autoreload 2

r = re.compile('.*maoz*.')
if len(list(filter(r.match, sys.path))) != 0:
    PHYTECH_DRIVE_PATH = os.environ['PHYTECH_DRIVE_PATH']
    PHYTECH_DRIVE_PATH = '/Users/maozdotan/Google Drive/My Drive'
    GITHUB_PATH = PHYTECH_DRIVE_PATH + '/GitHub'
    DATA_WD = PHYTECH_DRIVE_PATH + '/Data Integrity/SM anomalies/data'
    CERT_PATH = PHYTECH_DRIVE_PATH + '/Data'
    if CERT_PATH not in sys.path:
        sys.path.append(CERT_PATH)
    from sql_import_export import SqlImporter
    import cert_aws as c
    sql_importer = SqlImporter(database = c.database_research, user = c.user_research, password = c.password_research,
                                host = c.host_research, port = c.port_research, verbose=True)

local aws_cert.py loaded


In [7]:
%reload_ext autoreload
import project_class_data_extract
import logic_parameters# import default_latitude, default_height
from tqdm import tqdm_notebook
from functions import find_not_responding_events,get_project_results,load_project_data,not_responding_logic
#import common_db as cdb

In [ ]:
if (min(df_irr_span_short.sm_val) > probeMaxSM - 0.5) and
                ((max(df_irr_span_short.sm_diff) > sm_hourly_diff*logic_parameters.LOW_RESPONSE_FACTOR)):

- You call the 'Project' class
- you bring in the data: project.load_sm_project_data
- you apply transformers on the data (to fill missing values, for example): project.apply_transformers()
 - and then you group the TS by depths:
    project.group_data_to_depths()

In [50]:
SM_HOURLY_DIFF_FIRST_DEPTH = 0.5
SM_HOURLY_DIFF_SECOND_DEPTH = 0.25
MIN_IRR_AMOUNT = 1
MAX_RESPONDING_EVENTS = 1
LOW_RESPONSE_FACTOR = 0.5
IRR_SPAN_END_AFTER_X_HOURS = 4


#project_list = [852093, 871812, 851995, 852015]
summary_df = pd.DataFrame()
yesterday = (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d")
start_date = (dt.date.today() - dt.timedelta(days=8)).strftime("%Y-%m-%d")
failed_list = []

In [22]:
#project_results = find_not_responding_events(project_data)

query = f"""
    SELECT distinct(pm.project_id)
    FROM projects_metadata pm
    JOIN soil_sensors_metadata ssm
    ON pm.project_id = ssm.project_id
    JOIN project_irrigation_spans_v2 pis
    ON pis.project_id = pm.project_id
    WHERE time_zone like '%Los_Angeles'
    AND season = 2023
    AND type_id IN (90, 91, 92, 98, 117, 118, 124, 127, 135, 137)
    AND start_date >= CAST((CAST('{start_date}' AS timestamp)) AS date)
    AND active=true
    ORDER BY project_id
    """
sql_importer = SqlImporter(query=query, database=c.database_production, user=c.user_production, password=c.password_production,
                                host=c.host_production, port=c.port_production, verbose=True)

sql_importer.get_data()
project_list = sql_importer.data
project_list

Loaded table with 2587 lines from projects_metadata


,project_id
0,848545
1,848547
2,849306
3,849307
4,849370
...,...
2582,878567
2583,879153
2584,879190
2585,879195


In [53]:
summary_df = pd.DataFrame()
yesterday = (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d")
start_date = (dt.date.today() - dt.timedelta(days=8)).strftime("%Y-%m-%d")
failed_list = []

#project_list = [852093, 871812, 851995, 852015]
for p_id in project_list['project_id'][:-300]:
    #if p_id != 848545:
    #    continue
    try:
        project_data = load_project_data(project_id=p_id, min_date=start_date,
                      max_date=yesterday, min_depth=10, max_depth=91, debug=False)
        if (project_data.df_irrigation.empty):
                continue
        if project_data.valid_project:
            project_df = get_project_results(project_data)
            summary_df = pd.concat([summary_df,project_df],axis=0)
            #display(project_df, summary_df)
        else:
            failed_list.append(p_id)
    except Exception as e:
        print(p_id, e)
        failed_list.append(p_id)
        pass

display(summary_df)
#display(summary_df[(summary_df.not_responding_events_count / summary_df.irrigation_events) > 0.5])

850176 'Project' object has no attribute 'df_irrigation'
850664 min() arg is an empty sequence
850705 'Project' object has no attribute 'df_irrigation'
850706 'Project' object has no attribute 'df_irrigation'
850708 'Project' object has no attribute 'df_irrigation'
850731 index out of bounds
850735 index out of bounds
851051 'Project' object has no attribute 'df_irrigation'
851156 'Project' object has no attribute 'df_irrigation'
851158 index out of bounds
851190 min() arg is an empty sequence
851321 'Project' object has no attribute 'df_irrigation'
851329 min() arg is an empty sequence
851635 'Project' object has no attribute 'df_irrigation'
851792 index out of bounds
851798 index out of bounds
851799 index out of bounds
851864 index out of bounds
851904 min() arg is an empty sequence
851909 min() arg is an empty sequence
851916 index out of bounds
851940 index out of bounds
851959 min() arg is an empty sequence
852021 min() arg is an empty sequence
852039 min() arg is an empty sequen

857636 'Project' object has no attribute 'df_irrigation'
857638 'Project' object has no attribute 'df_irrigation'
857653 index out of bounds
857657 'Project' object has no attribute 'df_irrigation'
857660 'Project' object has no attribute 'df_irrigation'
857661 'Project' object has no attribute 'df_irrigation'
857667 'Project' object has no attribute 'df_irrigation'
857669 index out of bounds
857671 'Project' object has no attribute 'df_irrigation'
857695 min() arg is an empty sequence
857864 'Project' object has no attribute 'df_irrigation'
857868 'Project' object has no attribute 'df_irrigation'
857869 'Project' object has no attribute 'df_irrigation'
857871 'Project' object has no attribute 'df_irrigation'
857872 'Project' object has no attribute 'df_irrigation'
857882 'Project' object has no attribute 'df_irrigation'
857901 'Project' object has no attribute 'df_irrigation'
857904 'Project' object has no attribute 'df_irrigation'
857905 'Project' object has no attribute 'df_irrigati

,project_id,sensor_id,area_id,area_name,company_name,crop_name,variety_id,probe_depths,irrigation_events,not_responding_events_count,max_SM_diff,SM_statistics,event_timestamp,support_status,support_updated_at,days_since_task_complete,remarks,link,timezone
0,848545,948299,13909,Orland,16453,almond,38,"[16.0, 31.0]",4,1,3.66,"{16.0: {'mean': 17.71143750000001, 'std': 2.73...","{2023-05-09 20:05:00, 2023-05-09 08:05:00, 202...",None,NaT,None,{2023-05-05 07:49:00: probe 31.0 Not respondin...,https://app.phytech.com/13909/78720/848545,America/Los_Angeles
0,848547,974505,13909,Orland,16453,almond,89,"[16.0, 31.0]",1,0,0.34,"{16.0: {'mean': 26.058571428571433, 'std': 0.5...",{2023-05-09 13:05:00},None,NaT,None,{2023-05-09 13:05:00: probe 16.0 Not respondin...,https://app.phytech.com/13909/78721/848547,America/Los_Angeles
0,849306,959916,14032,Crimson Ridge,14660,grape,55,"[16.0, 31.0]",0,0,None,NaN,None,None,NaT,None,None,https://app.phytech.com/14032/78527/849306,America/Los_Angeles
0,849307,966444,14032,Crimson Ridge,14660,grape,55,"[16.0, 31.0]",0,0,None,NaN,None,None,NaT,None,None,https://app.phytech.com/14032/78539/849307,America/Los_Angeles
0,849370,935728,3626,BP - Ranch 3,1002,almond,40,"[16.0, 31.0]",5,2,1.12,"{16.0: {'mean': 31.62187845303869, 'std': 0.92...","{2023-05-10 07:21:00, 2023-05-08 07:29:00, 202...",completed,2023-04-26 20:17:34.897115,17,{2023-05-11 07:11:00: probe 31.0 Not respondin...,https://app.phytech.com/3626/79603/849370,America/Los_Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,871041,805284,9649,Buccaneer,2486,valencia,304,"[15.0, 30.0]",2,0,3.42,"{15.0: {'mean': 34.452956989247305, 'std': 1.0...",{},closed,2023-02-08 04:28:26.459155,94,{},https://app.phytech.com/9649/7284/871041,America/Los_Angeles
0,871045,843199,9702,Konda 2,2486,navel,287,"[16.0, 31.0]",2,0,2.83,"{16.0: {'mean': 33.53091891891893, 'std': 0.79...",{},closed,2023-05-05 01:11:25.414904,8,{2023-05-11 07:06:00: probe 16.0 Low sensor re...,https://app.phytech.com/9702/7166/871045,America/Los_Angeles
0,871046,901078,9694,Ito 2,2485,mandarin,158,"[16.0, 31.0]",3,0,16.43,"{16.0: {'mean': 18.986413043478265, 'std': 1.1...",{},new,2023-05-05 00:39:31.182897,8,{},https://app.phytech.com/9694/7318/871046,America/Los_Angeles
0,871047,867294,9702,Konda 2,2486,navel,95,"[16.0, 31.0]",1,0,2.29,"{16.0: {'mean': 34.086573033707836, 'std': 0.8...",{},new,2023-05-05 00:38:32.753539,8,{2023-05-08 06:50:00: probe 31.0 Local saturat...,https://app.phytech.com/9702/7160/871047,America/Los_Angeles


In [54]:
df = summary_df[summary_df.irrigation_events!=0].copy()
#summary_df.groupby('project_id').count()
#df[df.project_id==848545]
#display(df[(df.not_responding_events_count / df.irrigation_events) > 0.5])
summary_df.shape

(1733, 19)

In [30]:
summary_df['percent_not responding'] = summary_df.apply(lambda row: (row.not_responding_events_count / row.irrigation_events) if row.irrigation_events>0 else 0, axis=1)
summary_df.to_csv(DATA_WD+'/summary_df3.csv')

In [41]:
proj_data = load_project_data(project_id=849883, min_date=start_date,
                      max_date=yesterday, min_depth=10, max_depth=91, debug=False)


In [46]:
proj_data.local_saturation_by_depth[0][1]
proj_data.SM_statistics[16.0]['max']

29.97

29.97

In [14]:
df_project = test_project_data(858805, debug=True, sql_debug=False)
df_project
df_project['remarks'][0]
df_project['SM_statistics'][0]

https://app.phytech.com/8610/47932/858805


,project_id,amount,start_ts,end_ts,psi,start_date,start,end,system_type
0,858805,1.834727,1683752400000,1683759000000,31.182016,2023-05-10,2023-05-10 14:00:00,2023-05-10 15:50:00,drip
1,858805,1.567857,1683846000000,1683851640000,30.456853,2023-05-11,2023-05-11 16:00:00,2023-05-11 17:34:00,drip


irrigation event: project_id                  858805
amount                     1.83473
start_ts             1683752400000
end_ts               1683759000000
psi                         31.182
start_date              2023-05-10
start          2023-05-10 14:00:00
end            2023-05-10 15:50:00
system_type                   drip
Name: 0, dtype: object
2023-05-10 14:00:00

probe_depth 15.0
            local_time  sm_val  sm_diff        date
0  2023-05-10 13:00:00   16.40    -0.03  2023-05-10
1  2023-05-10 14:00:00   16.40     0.00  2023-05-10
2  2023-05-10 15:00:00   16.36    -0.04  2023-05-10
3  2023-05-10 16:00:00   16.36     0.00  2023-05-10
4  2023-05-10 17:00:00   16.36     0.00  2023-05-10
5  2023-05-10 18:00:00   16.36     0.00  2023-05-10
6  2023-05-10 19:00:00   16.36     0.00  2023-05-10
7  2023-05-10 20:00:00   16.36     0.00  2023-05-10
8  2023-05-10 21:00:00   16.36     0.00  2023-05-10
9  2023-05-10 22:00:00   16.40     0.04  2023-05-10
10 2023-05-10 23:00:00   16.40    

,project_id,sensor_id,area_id,area_name,company_name,crop_name,variety_id,probe_depths,irrigation_events,not_responding_events_count,max_SM_diff,SM_statistics,event_timestamp,support_status,support_updated_at,days_since_task_complete,remarks,link,timezone
0,858805,1167657,8610,WSU - Sunrise Orchard,8641,apple,143,"[15.0, 30.0]",2,2,0.11,"{15.0: {'mean': 16.61176795580111, 'std': 0.26...","{2023-05-10 14:00:00, 2023-05-11 16:00:00}",completed,2023-05-04 03:10:16.754707,10,{2023-05-11 16:00:00: probe 30.0 Not respondin...,https://app.phytech.com/8610/47932/858805,America/Los_Angeles


{'2023-05-10 14:00:00: probe 15.0 Not responding to irrigation',
 '2023-05-10 14:00:00: probe 30.0 Not responding to irrigation',
 '2023-05-11 16:00:00: probe 15.0 Not responding to irrigation',
 '2023-05-11 16:00:00: probe 30.0 Not responding to irrigation'}

{15.0: {'mean': 16.61176795580111,
  'std': 0.2696297281473203,
  'max': 17.18,
  'min': 16.28},
 30.0: {'mean': 16.005469613259667,
  'std': 0.1999789583774714,
  'max': 16.38,
  'min': 15.74}}

In [8]:
def test_project_data(p_id, debug=False, sql_debug=False):
    yesterday = (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d")
    start_date = (dt.date.today() - dt.timedelta(days=8)).strftime("%Y-%m-%d")
    try:
        project_data = load_project_data(project_id=p_id, min_date=start_date,
                      max_date=yesterday, min_depth=10, max_depth=91, debug=sql_debug)
    except Exception as e:
        print(e)
        return
    print(project_data.app_link)
    if project_data.valid_project & debug:
        display(project_data.df_irrigation[project_data.df_irrigation.amount > logic_parameters.MIN_IRR_AMOUNT])

    if project_data.valid_project:
        projects_df = get_project_results(project_data, debug)
        return(projects_df)
    else:
        return('project_data.valid_project', project_data.valid_project)

def find_not_responding_events(project_data, debug=False):
    # initialize
    not_responding_SM_sensors_project_dict,probe_events_dict = {},{}
    event_timestamp = set()
    probe_depths = project_data.df_sm_data.depth_cm.unique()
    total = 0
    project_remarks = set()
    not_responding_SM_sensors_project_dict.update({'probe_depths': list(probe_depths),
                                                   'events_count': total,
                                                   'events_details': {},
                                                  'remarks': project_remarks})
    
    
    df_irr = project_data.df_irrigation[project_data.df_irrigation.amount > MIN_IRR_AMOUNT]
    for irr_event_counter,row in df_irr.iterrows():
        if debug:
            print(f"irrigation event: {row}")
        current_event_start = df_irr.start[irr_event_counter]
        not_responding = False
        
        if row['amount'] > MIN_IRR_AMOUNT:
            for probe_depth in probe_depths:
                counter=0
                ProbeMinMoisture, ProbeMaxMoisture = 0,0
                probe_events_list = []

                if debug:
                    print("\nprobe_depth", probe_depth)

                probe_dict = {}
                probe_depth_index = list(probe_depths).index(probe_depth)

                probe_local_saturation = project_data.local_saturation_by_depth[probe_depth_index][1]
                if project_data.multi_depths_sm[probe_depth].empty: # no SM data for this depth
                    project_remarks.add(f"missing SM data for {probe_depth}")
                    probe_dict.update({'irrigation_events': len(df_irr),
                                        'events dates': [],
                                        'probe SM diff': 0,
                                        'remarks': project_remarks})
                    probe_events_dict[probe_depth] = probe_dict
                    continue # check next depth

                df = project_data.multi_depths_sm[probe_depth].reset_index(drop=False)
                df['date'] = df.local_time.dt.date
                #################################
                # Not responding conditions:
                #
                # time frame irrigation span -1hr/ +4hr
                # soil moisture hourly diff > SM_HOURLY_DIFF according to depth
                # soil moisture hourly diff < SM_HOURLY_DIFF * 0.5 = Low sensor respnse
                # initial probe moisture is less than local saturation minus 0.5%
                #################################
                df_irr_span = df[(df.local_time > current_event_start - pd.Timedelta(hours=1)) 
                                 & (df.local_time < df_irr.end[irr_event_counter] + pd.Timedelta(hours=IRR_SPAN_END_AFTER_X_HOURS))
                                ]
                df_irr_wide_span = df[(df.local_time > current_event_start - pd.Timedelta(hours=1)) 
                                 & (df.local_time < df_irr.end[irr_event_counter] + pd.Timedelta(hours=2*IRR_SPAN_END_AFTER_X_HOURS))
                                ]
                #if debug:
                #    print(current_event_start,df_irr.end[irr_event_counter],df_irr_span)
                ProbeMinMoisture, ProbeMaxMoisture = min(df_irr_span.sm_val), max(df_irr_span.sm_val)
                
                SM_HOURLY_DIFF = SM_HOURLY_DIFF_FIRST_DEPTH if probe_depth_index==0 else SM_HOURLY_DIFF_SECOND_DEPTH
                #probe_dict['remarks'] = project_remarks
                # check if probe responding = at least one hourly diff above SM_HOURLY_DIFF
                #if (probe_depth_index==0) & (not_responding):
                if max(df_irr_span.sm_diff) < SM_HOURLY_DIFF:
                        # check if the probe initial moisture is near local saturation 
                    #if ("not_responding" in project_data.flag) | (max(df_irr_span.sm_val.iloc[:3]) < probe_local_saturation - 0.5):
                    if max(df_irr_span.sm_val.iloc[:3]) < (probe_local_saturation - 0.5):
                        counter+=1 # count number of not responding probes

                        not_responding = True
                        probe_events_list.append((probe_depth,current_event_start))
                        event_timestamp.add(current_event_start)
                        #probe_dict['remarks'] = 'Not responding to irrigation'
                    # Find sensors with low response (low peak in sensor graph)
                        if (max(df_irr_span.sm_diff) > SM_HOURLY_DIFF*LOW_RESPONSE_FACTOR) & (max(df_irr_span.sm_diff) < SM_HOURLY_DIFF):
                            #probe_dict['remarks'] += ',Low sensor respnse'
                            project_remarks.add(f"{current_event_start}: probe {probe_depth} Low sensor respnse")
                        else:
                            project_remarks.add(f"{current_event_start}: probe {probe_depth} Not responding to irrigation")
                    else:
                        #probe_dict['remarks'] += f",probe {probe_depth} local_saturation in irrigation event {irr_event_counter}"
                        project_remarks.add(f"{current_event_start}: probe {probe_depth} local_saturation")

                    #project_remarks.add(probe_dict['remarks'])
                    #probe_dict['remarks'] = project_remarks
                    probe_dict['irrigation_events'] = len(df_irr)
                    probe_dict['events dates'] = probe_events_list
                    probe_dict['probe SM diff'] = ProbeMaxMoisture - ProbeMinMoisture
                    probe_events_dict[probe_depth] = probe_dict
                
                else:
                    if max(df_irr_wide_span.sm_diff) > SM_HOURLY_DIFF:
                    #probe_dict['remarks'] += ',late sensor response'
                        project_remarks.add(f"{current_event_start}: probe {probe_depth} late sensor response")

                    if debug:
                        print(f"{current_event_start}: responding well")
                    #project_remarks.add(probe_dict['remarks'])
                    probe_dict.update({'irrigation_events': len(df_irr),
                                        'events dates': [],
                                        'probe SM diff': max(df_irr_span.sm_val) - min(df_irr_span.sm_val),
                                        #'remarks': probe_dict['remarks']
                                      'remarks':project_remarks})
                    probe_events_dict[probe_depth] = probe_dict
                    
                if debug:
                    print(f"max hourly diff: {max(df_irr_span.sm_diff)}",
                            f"probe local saturation: {probe_local_saturation}",
                            f"initial probe SM: {max(df_irr_span.sm_val.iloc[:3])}")
            
            # count number of not responding probes per irrigation span
            if counter == len(probe_depths): # all probes are NOT responding in this irrigation span
                probe_dict['remarks'] += f"{current_event_start}: All probes not responding"
            # First probe responded well: ignore other not_responding events
            elif (probe_events_dict[probe_depths[0]]['events dates']==[]):
                not_responding = False

            if not_responding:
                total+=1
            
            if debug:
                print(f"""total: {total} ,{current_event_start} finished\n######################""")
            not_responding_SM_sensors_project_dict.update({'probe_depths': list(probe_depths),
                                                           'events_count': total,
                                                           'event_timestamp': event_timestamp,
                                                           'events_details': probe_events_dict,
                                                          'remarks': project_remarks})
        else: # irrigation event amount less than minimum
            continue

    return(not_responding_SM_sensors_project_dict)

In [ ]:
def get_project_results(project_data, debug=False):
    project_results = find_not_responding_events(project_data,debug)
    df_irr = project_data.df_irrigation[project_data.df_irrigation.amount > MIN_IRR_AMOUNT]

    projects_df = pd.DataFrame(columns=['project_id','sensor_id','probe_depths','irrigation_events',
                                      'not_responding_events_count','max_SM_diff', 'event_timestamp',
                                       'support_status', 'support_updated_at','days_since_task_complete',
                                        'remarks', 'link','timezone'])
    project_dict = {}

    remarks = set()
    
    project_dict.update({'project_id' : project_data.project_id,
                         'sensor_id' : project_data.sensor_id,
                         'probe_depths': project_results.get('probe_depths'),
                         'irrigation_events' : len(df_irr),
                         'not_responding_events_count' : project_results['events_count'], #max_val,
                         'link' : project_data.app_link,
                         'timezone' : project_data['timezone']})
    
    # No not_responding events found
    if not project_results['events_details']: # empty dict = No events
        project_dict.update({'max_SM_diff': None,
                             #'probe_depths': None,
                             'event_timestamp' : None,
                             'remarks' : None})
    else:
        # find probe with max not responding events
        for d in project_results.get('probe_depths'):
            #remarks.add(project_results.get('events_details')[d]['remarks'])
            project_dict.update({'max_SM_diff': max(d['probe SM diff'] for d in project_results['events_details'].values()),
                                 #'probe_depths': project_results.get('probe_depths'),
                                 'event_timestamp' : project_results.get('event_timestamp'),
                                 #'remarks' : remarks
                                'remarks': project_results['remarks']})
    # Find support status information
    if not project_data.sensor_support_status:
        sensor_status = {'status': None, 'updated_at': None, 'days_since_task_complete': None}
    else:
        sensor_status = project_data.sensor_support_status_dict[0]
        sensor_status['days_since_task_complete'] = (dt.datetime.today().date() -
                                    sensor_status['updated_at'].date()).days
    
    project_dict.update({'support_status' : sensor_status['status'],
                         'support_updated_at' : sensor_status['updated_at'],
                         'days_since_task_complete': sensor_status['days_since_task_complete']})
    
    
    projects_df = pd.concat([projects_df, pd.DataFrame.from_dict(project_dict,orient='index').T], ignore_index=True)
    #projects_df['days_since_task_complete'] =  (dt.datetime.today().date() - projects_df['support_updated_at'][0].date()).days
    cols = ['project_id', 'sensor_id', 'probe_depths', 'irrigation_events',
       'not_responding_events_count', 'max_SM_diff', 'event_timestamp',
       'support_status', 'support_updated_at','days_since_task_complete', 'remarks', 'link', 'timezone']
    projects_df = projects_df[cols]
    
    return(projects_df)

In [14]:
def find_not_responding_events_old(project_data, debug=False):
    not_responding_SM_sensors_project_dict,probe_events_dict = {},{}
    event_timestamp = set()
    probe_depths = project_data.df_sm_data.depth_cm.unique()
    for probe_depth in probe_depths:
        probe_events_list = []
        
        if debug:
            print("\nprobe_depth", probe_depth)
            
        probe_dict = {}
        counter = 0
        probe_depth_index = list(probe_depths).index(probe_depth)
        
        for irr_event_counter,row in project_data.df_irrigation.iterrows():
            #probe_depth = project_data.df_sm_data.depth_cm.unique()[0]
            probe_local_saturation = project_data.local_saturation_by_depth[probe_depth_index][1]
            df = project_data.multi_depths_sm[probe_depth].reset_index(drop=False)
            df['date'] = df.local_time.dt.date

            if row['amount'] > MIN_IRR_AMOUNT:
                #################################
                # Not responding conditions:
                #
                # time frame irrigation span -1hr/ +3hr
                # soil moisture hourly diff > SM_HOURLY_DIFF according to depth
                # initial probe moisture is less than local saturation minus 0.5%
                #################################
                df_irr_span = df[(df.local_time > project_data.df_irrigation.start[irr_event_counter] - pd.Timedelta(hours=1)) 
                                 & (df.local_time < project_data.df_irrigation.end[irr_event_counter] + pd.Timedelta(hours=4))
                                ]
                if debug:
                    display(project_data.df_irrigation.start[irr_event_counter],project_data.df_irrigation.end[irr_event_counter],
                            df_irr_span)
                
                
                SM_HOURLY_DIFF = SM_HOURLY_DIFF_FIRST_DEPTH if probe_depth_index==0 else SM_HOURLY_DIFF_SECOND_DEPTH
                probe_dict['remarks'] = ''
                # check if probe responding = at least one hourly diff above SM_HOURLY_DIFF
                if max(df_irr_span.sm_diff) < SM_HOURLY_DIFF:
                    # check if the probe initial moisture is near local saturation 
                    if ("not_responding" in project_data.flag) | (max(df_irr_span.sm_val.iloc[:3]) < probe_local_saturation - 0.5):
                        counter+=1 # Not responding to irrigation
                        not_responding_sensor_id = int(project_data.df_sm_data[project_data.df_sm_data['depth_cm']==probe_depth].loc[0,'sensor_id'])
                        probe_events_list.append((probe_depth,project_data.df_irrigation.start[irr_event_counter]))
                        event_timestamp.add(project_data.df_irrigation.start[irr_event_counter])
                        probe_dict['remarks'] = 'Not responding to irrigation'
                # Find sensors with low response (short peak in sensor graph)
                    if (max(df_irr_span.sm_diff) > SM_HOURLY_DIFF*LOW_RESPONSE_FACTOR) & (max(df_irr_span.sm_diff) < SM_HOURLY_DIFF):
                        probe_dict['remarks'] = 'Low sensor respnse'

                if debug:
                    display(f"max hourly diff: {max(df_irr_span.sm_diff)}",
                            f"probe local saturation: {probe_local_saturation}",
                            f"initial probe moisture: {max(df_irr_span.sm_val.iloc[:3])}")
                    
            probe_dict['irrigation_events'] = len(project_data.df_irrigation)
            probe_dict['not_responding_events_count'] = counter
            probe_dict['events dates'] = probe_events_list
            probe_events_dict[probe_depth] = probe_dict

    not_responding_SM_sensors_project_dict.update({'probe_depths':list(probe_depths),
                                                 'event_timestamp':event_timestamp,
                                                 'events_details':probe_events_dict})
    return(not_responding_SM_sensors_project_dict)



In [412]:
query = f"""
            SELECT serial_number as sensor_id, updated_at, status
            FROM work_order_line_items
            WHERE serial_number = '{project_data.sensor_id}'
            --AND status not in ('closed','completed')
            order by updated_at desc
            limit 1
        """
sql_importer = SqlImporter(query=query, database=c.database_ruby_production, user=c.user_production, password=c.password_production, host=c.host_production, port=c.port_production, verbose=True)

sql_importer.get_data()
sql_importer.data

Loaded table with 1 lines from work_order_line_items



,sensor_id,updated_at,status
0,934741,2023-04-27 08:52:33.235331,new


def load_project_data(project_id, min_date, max_date, min_depth=10, max_depth=91, debug=False):
    from logic_parameters import default_latitude, default_height,default_max_depth
    from project_class_data_extract import Project
    project = Project(
        project_id=project_id,
        min_depth=min_depth,
        max_depth=default_max_depth,  # set the depth range we're interested in
        min_date=min_date,
        max_date=max_date,
        debug=debug)

    project.load_project_metadata()
    project.get_sm_depths()

    if len(project.depths_found) == 0:
        project.valid_project = False
        return project

    project.load_sm_project_data(min_depth=project.depths_found[0], max_depth=project.depths_found[-1], )  # change the min/max_depth if you dont want to load all depths
    if not project.valid_project:
        return project
    #project.load_project_weather_data(future=14)  # load the weather date until max_date + 14 days.
    project.apply_transformers()
    project.group_data_to_depths()
    project.load_irrigation_spans()
    project.find_probe_local_saturation()

    project.meta_data = {'project_id': project.project_id, 'latitude': project.latitude if project.latitude else default_latitude,
                         'height': project.height if project.height else default_height, 'app_link': project.app_link}
    project.sensor_support_status_dict = project.get_sensor_support_status()

    return project

In [29]:
def transform_data(project_data, depth, max_datetime, max_history_days):
    from config.logic_parameters import max_hours_after_peak, rise_th, peak_th
    dt = DataTransformer(project_data['meta_data'], project_data['multi_depths_sm'], project_data['weather_data'], depth=depth, max_history_days=max_history_days,
                         max_datetime=max_datetime, rise_th=rise_th, peak_th=peak_th)

    dt.collect_historical_peak2rise_events()
    dt.calc_sm_diffs()
    dt.extract_data_from_p2r(max_hours_after_peak=max_hours_after_peak)
    dt.add_lower_depth_data()
    dt.add_weather_data()
    project_data.update({'model_data': dt.data})

    model_data = dict(meta_data=project_data['meta_data'], model_data=project_data['model_data'])
    run_data = {'depth': dt.depth, 'max_datetime': dt.max_datetime, 'max_history_days': dt.max_history_days}
    model_data.update(run_data)
    return model_data

In [247]:
def load_irrigation_spans(p_id):
        query = f"""
            SELECT project_id, amount, start_ts, end_ts, psi, start_date, 
            timezone('America/Los_Angeles', to_timestamp(start_ts / 1000)) start_lt,
            timezone('America/Los_Angeles', to_timestamp(end_ts / 1000)) end_lt
            FROM project_irrigation_spans_v2
            WHERE project_id = {p_id} 
            AND start_date >= CAST((CAST('2023-05-02' AS timestamp)) AS date)
            AND start_date <= CAST((CAST('2023-05-09' AS timestamp)) AS date)
        """
        query=f"""
        SELECT pis.project_id, amount, start_ts, end_ts, psi, start_date, 
            timezone('America/Los_Angeles', to_timestamp(start_ts / 1000)) start_lt,
            timezone('America/Los_Angeles', to_timestamp(end_ts / 1000)) end_lt,
            irrigation_system_type system_type
            FROM project_irrigation_spans_v2 pis
            JOIN  projects_hierachy ph
            ON pis.project_id = ph.project_id
            WHERE pis.project_id = ({p_id}) OR 
                    pis.project_id = (SELECT project_id from projects_hierachy
                    WHERE  main_project_const_id = (SELECT main_project_const_id FROM projects_hierachy WHERE project_id = {p_id})
                    AND season = 2023
                    AND project_id <> {p_id})
            AND start_date >= CAST((CAST('2023-05-02' AS timestamp)) AS date)
            AND start_date <= CAST((CAST('2023-05-09' AS timestamp)) AS date)
            ORDER BY start_date
        """

        sql_importer = SqlImporter(query=query, database=c.database_production, user=c.user_production, password=c.password_production,
                                host=c.host_production, port=c.port_production, verbose=True)

        sql_importer.get_data()
        print(sql_importer.query)
        df_irrigation = sql_importer.data
        display(df_irrigation)#.rename(columns={'start_lt': 'start', 'end_lt': 'end'}, inplace=True))

load_irrigation_spans(855929)

Loaded table with 21 lines from project_irrigation_spans_v2

        SELECT pis.project_id, amount, start_ts, end_ts, psi, start_date, 
            timezone('America/Los_Angeles', to_timestamp(start_ts / 1000)) start_lt,
            timezone('America/Los_Angeles', to_timestamp(end_ts / 1000)) end_lt,
            irrigation_system_type system_type
            FROM project_irrigation_spans_v2 pis
            JOIN  projects_hierachy ph
            ON pis.project_id = ph.project_id
            WHERE pis.project_id = (855929) OR 
                    pis.project_id = (SELECT project_id from projects_hierachy
                    WHERE  main_project_const_id = (SELECT main_project_const_id FROM projects_hierachy WHERE project_id = 855929)
                    AND season = 2023
                    AND project_id <> 855929)
            AND start_date >= CAST((CAST('2023-05-02' AS timestamp)) AS date)
            AND start_date <= CAST((CAST('2023-05-09' AS timestamp)) AS date)
            ORDER B

,project_id,amount,start_ts,end_ts,psi,start_date,start_lt,end_lt,system_type
0,855929,1.140883,1680813300000,1680816300000,35.968093,2023-04-06,2023-04-06 13:35:00,2023-04-06 14:25:00,drip
1,855929,1.346242,1680877800000,1680881340000,37.708484,2023-04-07,2023-04-07 07:30:00,2023-04-07 08:29:00,drip
2,855929,1.597237,1681404600000,1681408800000,41.334300,2023-04-13,2023-04-13 09:50:00,2023-04-13 11:00:00,drip
3,855929,2.395855,1681848900000,1681855200000,43.219724,2023-04-18,2023-04-18 13:15:00,2023-04-18 15:00:00,drip
4,855929,1.140883,1682095200000,1682098200000,41.334300,2023-04-21,2023-04-21 09:40:00,2023-04-21 10:30:00,drip
5,855929,1.140883,1682114100000,1682117100000,41.914431,2023-04-21,2023-04-21 14:55:00,2023-04-21 15:45:00,drip
6,855929,2.281767,1682171400000,1682177400000,34.807832,2023-04-22,2023-04-22 06:50:00,2023-04-22 08:30:00,drip
7,855929,0.000000,1682524200000,1682524200000,40.754170,2023-04-26,2023-04-26 08:50:00,2023-04-26 08:50:00,drip
8,855929,1.369060,1682607000000,1682610600000,43.074692,2023-04-27,2023-04-27 07:50:00,2023-04-27 08:50:00,drip
9,855929,1.140883,1682977800000,1682980800000,35.678028,2023-05-01,2023-05-01 14:50:00,2023-05-01 15:40:00,drip
